In [1]:
!pip install faiss-gpu langchain-cohere langchain llama-index langchain-core chromadb langchain-chroma streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
path= drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
from google.colab import userdata
my_secret_key = userdata.get('COHERE_KEY')

In [15]:
# !pip install faiss-gpu langchain-cohere langchain llama-index langchain-core chromadb langchain-chroma streamlit

%%writefile app.py


import bs4
import os
import streamlit as st
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_cohere import ChatCohere
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_cohere.embeddings import CohereEmbeddings
from langchain_community.vectorstores import FAISS, Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext

st.title("Document QA System")

os.environ["COHERE_API_KEY"] = my_secret_key

folder_path= "/content/drive/MyDrive/data/Bajaj Finance.pdf"

reader = PyPDFLoader(folder_path)
doc = reader.load_and_split()

text_splitter= RecursiveCharacterTextSplitter(chunk_size =1000, chunk_overlap=10)
data=text_splitter.split_documents(doc)

llm = ChatCohere(temperature=0)

embeddings = CohereEmbeddings()

vectorstore = FAISS.from_documents(documents=data, embedding=embeddings)

retriever = vectorstore.as_retriever()

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

qa_log = []

st.write("Conversation:")
for i, (question, answer) in enumerate(qa_log, start=1):
    st.write(f"User: {question}")
    st.write(f"Assistant: {answer}")

user_question = st.text_input("Enter your question:")

if st.button("Get Answer"):
    answer = conversational_rag_chain.invoke(
        {"input": user_question},
        config={"configurable": {"session_id": "abc123"}}
    )["answer"]
    st.write("Answer:", answer)
    qa_log.append((user_question, answer))


Overwriting app.py


In [16]:
! wget -q -O - ipv4.icanhazip.com

34.125.24.77


In [17]:
! streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.24.77:8501

npx: installed 22 in 2.68s
your url is: https://shaky-melons-yell.loca.lt
Parent run 45297a98-96cb-405d-bdc7-eee9ad020f78 not found for run eef5c628-595b-4a6b-bb97-ea000a37abee. Treating as a root run.
Parent run bfab75f9-0252-4d17-aaaa-ea1542915944 not found for run 3c9df943-547f-43ff-b53c-d2f2ae49742a. Treating as a root run.
  Stopping...
^C
